In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
# ============================================================
# ✅ FIX PATH FOR NOTEBOOK INSIDE /notebooks
# ============================================================
import sys, os
from pathlib import Path

# Xác định thư mục cha của notebooks/
project_root = Path(os.getcwd()).parent  # => C:\Users\User\RR_model
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))  # ⚠️ thêm project_root, KHÔNG thêm src

print("📁 Project root added:", project_root)

# Kiểm tra lại
import importlib.util
print("🔍 src module found:", importlib.util.find_spec("src"))

📁 Project root added: C:\Users\MAFC4709\Python_work\RR_model_1911
🔍 src module found: ModuleSpec(name='src', loader=<_frozen_importlib_external.SourceFileLoader object at 0x000001F82322DB70>, origin='C:\\Users\\MAFC4709\\Python_work\\RR_model_1911\\src\\__init__.py', submodule_search_locations=['C:\\Users\\MAFC4709\\Python_work\\RR_model_1911\\src'])


In [3]:
# ===========================================================
# 📘 Roll Rate Demo v7 - Sử dụng dữ liệu thật từ Oracle hoặc SQL file
# ===========================================================

from pathlib import Path
import sys
import pandas as pd
import numpy as np

# --- 1️⃣ Setup đường dẫn và import các module ---
root = Path(".").resolve()
sys.path.append(str(root / "src"))

from src.data_loader import load_data
from src.config import CFG, OUT_ROOT
from src.rollrate.transition import compute_transition_from_pairs, compute_transition_by_mob

from src.rollrate.export_excel import export_transition_excel
from src.rollrate.backtest import (
    compute_actual_matrix,
    rollforward_validation,
    plot_matrix_diff,
    plot_distribution_compare
)

print("✅ Import modules thành công.")
print("Cấu hình CFG:", CFG)


# --- 2️⃣ Load dữ liệu thực tế ---
#sql_path = r"C:/Users/MAFC4709/Python_work/RR_model/sql/uns2.sql"

#df = load_data(sql_path)
df = load_data("C:/Users/MAFC4709/Python_work/RR_model_1911/data/parquet/output.parquet")
df['DISBURSAL_DATE'] = df['DISBURSAL_DATE'].apply(lambda x: x.replace(day=1))

#print(f"📦 Đã load {len(df):,} dòng dữ liệu từ {sql_path}")

# Kiểm tra các cột quan trọng
required_cols = [CFG["loan"], CFG["mob"], CFG["state"], CFG["ead"], CFG["cutoff"]]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"❌ Thiếu các cột cần thiết trong data: {missing}")

print("✅ Dữ liệu có đủ các cột cần thiết:", required_cols)

# Nếu chưa có PRODUCT_TYPE thì thêm mặc định
if "PRODUCT_TYPE" not in df.columns:
    df["PRODUCT_TYPE"] = "A"
    print("⚠️ Không có PRODUCT_TYPE trong data, gán mặc định = 'A'")

# Convert cutoff về datetime
df[CFG["cutoff"]] = pd.to_datetime(df[CFG["cutoff"]])





✅ Import modules thành công.
Cấu hình CFG: {'loan': 'AGREEMENT_ID', 'mob': 'MOB', 'state': 'STATE_MODEL', 'orig_date': 'DISBURSAL_DATE', 'ead': 'PRINCIPLE_OUTSTANDING', 'disb': 'DISBURSAL_AMOUNT', 'cutoff': 'CUTOFF_DATE'}
📦 Loading Parquet from: C:\Users\MAFC4709\Python_work\RR_model_1911\data\parquet\output.parquet
✅ Loaded 25,998,763 rows from output.parquet
✅ Dữ liệu có đủ các cột cần thiết: ['AGREEMENT_ID', 'MOB', 'STATE_MODEL', 'PRINCIPLE_OUTSTANDING', 'CUTOFF_DATE']


In [4]:
# ============================================================
# STEP 1 – TRANSITION MATRICES
# ============================================================
matrices_by_mob, parent_fallback = compute_transition_by_mob(df)
#print("✔ matrices_by_mob built")

# ============================================================


⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=500-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=550-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=600-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=650+)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=650-)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=CDLPIL, score=OTHERS)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 'PREPAY', 'SOLDOUT']
⚙️ Built parent fallback for (product=SALPIL, score=A)
⚠️ Có 4 hàng có tổng weight = 0: ['DPD120+', 'DPD180+', 

In [5]:
def add_del_metrics_simple(df_lifecycle: pd.DataFrame) -> pd.DataFrame:
    """
    Rất quan trọng: dùng cho forecast_long_df.
    Không có DISB_TOTAL từ df_raw, nên nếu VINTAGE trùng nhau
    → bạn cần có cột DISB_TOTAL trước đó.

    Nếu chưa có DISB_TOTAL → mặc định lấy tổng EAD tại MOB0.
    """

    df = df_lifecycle.copy()

    # Nếu chưa có DISB_TOTAL → tạo từ MOB0
    if "DISB_TOTAL" not in df.columns:
        df0 = (
            df[df["MOB"] == 0]
            .groupby(["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE"])[BUCKETS_CANON[0]]
            .sum()
            .rename("DISB_TOTAL")
        )

        df = df.merge(
            df0,
            on=["PRODUCT_TYPE", "RISK_SCORE", "VINTAGE_DATE"],
            how="left"
        )

    # Tính AMT
    df["DEL30_AMT"] = df[BUCKETS_30P].sum(axis=1)
    df["DEL60_AMT"] = df[BUCKETS_60P].sum(axis=1)
    df["DEL90_AMT"] = df[BUCKETS_90P].sum(axis=1)

    # Tính tỷ lệ
    df["DEL30_PCT"] = df["DEL30_AMT"] / df["DISB_TOTAL"]
    df["DEL60_PCT"] = df["DEL60_AMT"] / df["DISB_TOTAL"]
    df["DEL90_PCT"] = df["DEL90_AMT"] / df["DISB_TOTAL"]

    return df

In [6]:
from src.rollrate.forecast import forecast_all_vintages
from src.rollrate.lifecycle import (
    get_actual_all_vintages_amount,
    combine_all_lifecycle_amount,
    lifecycle_to_long_df_amount,
    tag_forecast_rows_amount,
    add_del_metrics,
    aggregate_to_product,
    aggregate_products_to_portfolio,
    export_lifecycle_all_products_one_file,
    extend_actual_info_with_portfolio,
    BUCKETS_30P, BUCKETS_60P, BUCKETS_90P,
)

In [7]:
# STEP 2 – FORECAST LỊCH SỬ (ENGINE CŨ) → LIFECYCLE BASE (EAD)
# ============================================================
forecast_results = forecast_all_vintages(
    df_raw=df,
    matrices_by_mob=matrices_by_mob,
    max_mob=29,
    enable_macro=False,
)

actual_results = get_actual_all_vintages_amount(df)
lifecycle_dict = combine_all_lifecycle_amount(actual_results, forecast_results)
df_lifecycle_base = lifecycle_to_long_df_amount(lifecycle_dict)

print("✔ df_lifecycle_base:", df_lifecycle_base.shape)

✔ df_lifecycle_base: (19170, 14)


In [9]:
# ============================================================
# LOAD MODULES
# ============================================================
from src.config import CFG, BUCKETS_CANON
from src.rollrate.lifecycle import add_del_metrics
#from src.rollrate.del_metrics_simple import add_del_metrics_simple
from src.rollrate.forecast_full_history import forecast_full_history
from src.rollrate.calibration import compute_k_per_product_ifrs_fullhistory #, apply_k_to_forecast
from src.rollrate.seasonality import build_seasonality, apply_seasonality_to_lifecycle, apply_seasonality_to_sale_plan
from src.rollrate.lifecycle import aggregate_to_product, aggregate_products_to_portfolio, export_lifecycle_all_products_one_file

sale_plan_df = pd.read_excel("C:/Users/MAFC4709/Python_work/RR_model_1911/sale_plan_5yF2.xlsx", sheet_name="SALE_PLAN")
# ============================================================
# 0) LOAD RAW DATA
# ============================================================

df_raw = df.copy()     # bạn thay bằng data thực



# ============================================================
# 1) ACTUAL LIFECYCLE
# ============================================================

df_actual = get_actual_all_vintages_amount(df_raw)
df_actual_long = lifecycle_to_long_df_amount(df_actual)
df_actual_long = add_del_metrics(df_actual_long, df_raw=df_raw)
df_actual_long["IS_FORECAST"] = 0


# ============================================================
# 2) FORECAST FULL HISTORY (BACKTEST)
# ============================================================

df_fc_hist = forecast_full_history(df_raw, matrices_by_mob, max_mob=36)
df_fc_hist = add_del_metrics_simple(df_fc_hist)
df_fc_hist["IS_FORECAST"] = 1


# ============================================================
# 3) CALIBRATION k PER PRODUCT
# ============================================================

k_dict = compute_k_per_product_ifrs_fullhistory(
    df_actual=df_actual_long,
    df_forecast=df_fc_hist,
    min_obs=5,
    method="trimmed_mean",
)

print(">> k per product:", k_dict)


# ============================================================
# 4) ÁP K cho forecast lịch sử & sale plan
# ============================================================

df_fc_hist_calib = apply_k_to_forecast(df_fc_hist, k_dict)

sale_plan_fc = forecast_sale_plan_by_mob(
    sale_plan_df=sale_plan_df,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
    mob_target=24,
    start_state=BUCKETS_CANON[0],
    states=BUCKETS_CANON,
)
sale_plan_fc = add_del_metrics_simple(sale_plan_fc)
sale_plan_fc = apply_k_to_forecast(sale_plan_fc, k_dict)


# ============================================================
# 5) SEASONALITY
# ============================================================

season = build_seasonality(df_actual_long)
df_fc_hist_season = apply_seasonality_to_lifecycle(df_fc_hist_calib, season)
sale_plan_fc_season = apply_seasonality_to_sale_plan(sale_plan_fc, season)


# ============================================================
# 6) COMBINE ACTUAL + FORECAST + SALE PLAN
# ============================================================

df_final = pd.concat(
    [df_actual_long, df_fc_hist_season, sale_plan_fc_season],
    ignore_index=True
)


# ============================================================
# 7) AGGREGATE PRODUCT + PORTFOLIO
# ============================================================

df_prod = aggregate_to_product(df_final)
df_port = aggregate_products_to_portfolio(df_prod)


# ============================================================
# 8) EXPORT
# ============================================================

export_lifecycle_all_products_one_file(
    df_del_prod=df_prod,
    actual_info={},
    filename="Lifecycle_Final.xlsx"
)

print("✔✔✔ DONE: Pipeline đã chạy 100% thành công ✔✔✔")


⚙️ forecast_full_history(): có 639 cohort để backtest
✅ forecast_full_history(): output shape = (23643, 14), MOB_min=0, MOB_max=36
[WARN] Product TWLPIL quá ít sample → k = 1.0
>> k per product: {'CDLPIL': 0.7589178427905829, 'SALPIL': 0.8582665752669982, 'SPLPIL': 0.3899935326607954, 'TOPUP': 0.878879545410113, 'TWLPIL': 1.0, 'XSELL': 0.6477172871002819}


NameError: name 'apply_k_to_forecast' is not defined

In [10]:
df_lifecycle_base

,PRODUCT_TYPE,RISK_SCORE,VINTAGE_DATE,MOB,DPD0,DPD1+,DPD30+,DPD60+,DPD90+,DPD120+,DPD180+,PREPAY,WRITEOFF,SOLDOUT
0,CDLPIL,500-,2023-01-01,0,493.374711,3.613740,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
1,CDLPIL,500-,2023-01-01,1,418.023702,21.318179,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
2,CDLPIL,500-,2023-01-01,2,308.596853,53.241728,18.335200,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
3,CDLPIL,500-,2023-01-01,3,265.708119,5.770333,28.830319,13.005200,0.000000,0.0,0.0,0.000000,0.000000,0.0
4,CDLPIL,500-,2023-01-01,4,180.169614,28.523850,5.770333,20.863773,17.520001,0.0,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19165,XSELL,D,2025-09-01,25,12960.646062,1404.289727,1404.570635,744.831512,2742.671920,0.0,0.0,147.622141,12909.705059,0.0
19166,XSELL,D,2025-09-01,26,12273.840800,1361.200280,1450.751230,822.741675,2676.603543,0.0,0.0,178.883358,13550.316171,0.0
19167,XSELL,D,2025-09-01,27,11624.882841,1308.619218,1449.920319,862.544953,2663.543262,0.0,0.0,208.708576,14196.117888,0.0
19168,XSELL,D,2025-09-01,28,11011.136400,1251.602885,1420.926154,871.901341,2673.478934,0.0,0.0,237.095635,14848.195709,0.0


In [11]:
forecast_results

{('CDLPIL',
  '500-',
  Timestamp('2023-01-01 00:00:00')): {24: STATE_MODEL
  DPD0         0.000000
  DPD1+        0.000000
  DPD30+       0.000000
  DPD60+       0.000000
  DPD90+       0.000000
  DPD120+      0.000000
  DPD180+      0.000000
  PREPAY       0.000000
  WRITEOFF    81.127901
  SOLDOUT      0.000000
  Name: PRINCIPLE_OUTSTANDING, dtype: float64, 25: DPD0         0.000000
  DPD1+        0.000000
  DPD30+       0.000000
  DPD60+       0.000000
  DPD90+       0.000000
  DPD120+      0.000000
  DPD180+      0.000000
  PREPAY       0.000000
  WRITEOFF    81.127901
  SOLDOUT      0.000000
  dtype: float64, 26: DPD0         0.000000
  DPD1+        0.000000
  DPD30+       0.000000
  DPD60+       0.000000
  DPD90+       0.000000
  DPD120+      0.000000
  DPD180+      0.000000
  PREPAY       0.000000
  WRITEOFF    81.127901
  SOLDOUT      0.000000
  dtype: float64, 27: DPD0         0.000000
  DPD1+        0.000000
  DPD30+       0.000000
  DPD60+       0.000000
  DPD90+       0.00